# Slice Builder: GraphML Parsing

## Pick a Slice Name, Site to Work on, and the GraphML File to Parse

In [ ]:
# Constants to change as needed
SLICE_NAME = "Clos"
SITE_NAME = "MASS"
GRAPH_PATH = "/home/fabric/work/custom/FABRIC-Automation/graphs/2tier_test.graphml"
HAS_CLIENTS = True
CLIENT_PREFIX = "C"

## Import the FABlib Library and Confirm the Configuration is Correct

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager


try: 
    fablib = fablib_manager()
                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

## Parse GraphML for Topology and Create Slice

In [ ]:
import xml.dom.minidom
from collections import Counter

STARTING_INTF_NUM = 1

try:        
    #Create the slice
    slice = fablib.new_slice(name=SLICE_NAME)
    
    # Create dictionary to store nodes
    nodeDict = {}
    
    # Use XML parser to parse the GraphML file
    docs = xml.dom.minidom.parse(GRAPH_PATH)

    # Find all nodes via the node tag, add each to the slice with Rocky Linux as its base
    nodes = docs.getElementsByTagName("node")
    for node in nodes:
        # Grab the node name and determine if it is a client node based on its name prefix
        nodeName = node.getAttribute("id")
        isClient = True if HAS_CLIENTS and nodeName.startswith(CLIENT_PREFIX) else False # Check for compute/client nodes
        
        # Add node to the slice
        nodeInfo = slice.add_node(name=nodeName, cores=1, ram=4, image='default_rocky_8', site=SITE_NAME)
        
        # nodeDict = 0 -> FABRIC node object, 1 -> current intf number, 2 -> is a client/server (True) or not (False)
        nodeDict[nodeName] = {"nodeInfo": nodeInfo, "currentIntfNum": STARTING_INTF_NUM, "isClient": isClient}
        #[nodeInfo, 1, isClient]
        
        print(f'Added node {nodeName}')
    
    # Find all edges via the edge tag, add each to the slice via an L2Bridge connecting the node interfaces
    edges = docs.getElementsByTagName("edge")
    for edge in edges:
        # grab nodes x and y in edge (x,y)
        source = edge.getAttribute("source")
        target = edge.getAttribute("target")
        
        # Create an interface name for each interface in the network
        sourceIntfName = "eth{num}".format(node=source, num=nodeDict[source]["currentIntfNum"])
        targetIntfName = "eth{num}".format(node=target, num=nodeDict[target]["currentIntfNum"])
        
        # Name a network based on if it is a user-facing LAN (edge) or P2P links in the core of the network (core)
        networkPrefix = "edge" if nodeDict[source]["isClient"]  or nodeDict[target]["isClient"] else "core"
        networkName = f'{networkPrefix}-{source}-{target}'
        
        # Add a NIC for each node that is a part of the edge
        sourceIntf = nodeDict[source]["nodeInfo"].add_component(model='NIC_Basic', name=sourceIntfName).get_interfaces()[0]
        targetIntf = nodeDict[target]["nodeInfo"].add_component(model='NIC_Basic', name=targetIntfName).get_interfaces()[0]
        
        nodeDict[source]["currentIntfNum"] += 1
        nodeDict[target]["currentIntfNum"] += 1

        # Add a L2 network between the interfaces
        slice.add_l2network(name=networkName, interfaces=[sourceIntf, targetIntf], type="L2Bridge")
        
        print(f'Added edge {source}-{target}')

     
    #Submit Slice Request
    slice.submit()

except Exception as e:
    print(f"Exception: {e}")

## Add Basic IPv4 Addressing

In this system, 192.168.0.0/16 is the address range for all interfaces on the FABRIC slice. Each network is a /24 subnet of this network. Edge networks have client/compute devices with lower address (ex: .1) and networking nodes with higher addresses.

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv4Network

# Start with a 1 in the third octet
thirdOctet = 1

for network in slice.get_networks():
    networkAddress = f'192.168.{thirdOctet}.0/24'
    currentIPNetwork = IPv4Network(networkAddress)
    hostIPList = list(currentIPNetwork)[1:-1]
    
    networkName = network.get_name()
    isEdgeNetwork = True if networkName.startswith("edge") else False
    
    print(f"Configuring network {networkName} with IPv4 Network {networkAddress}")
    
    for intf in network.get_interfaces():
        intfName = intf.get_device_name()
        nodeName = intf.get_node().get_name()
        
        # If this is an edge network and it is a networking node, not the client node, give it a high-range address.
        if(isEdgeNetwork and not nodeDict[nodeName]["isClient"]):
            currentIPv4Address = hostIPList.pop() # Highest available host address in network
        else:
            currentIPv4Address = hostIPList.pop(0) # Lowest available host address in network
        
        intf.ip_addr_add(addr=currentIPv4Address, subnet=currentIPNetwork)
        print(f"\t{nodeName} {intf.get_device_name()} = {currentIPv4Address}")
        
    thirdOctet += 1